In [ ]:
from eib import *

YMIN = 1980

TODO : 
* Update PP when available in v31
* update wind percentiles with v31
* Use 0.1° resolution?

In [ ]:
eib = xr.open_dataset("extended-ibtracs/extended-ibtracs_NA.nc")
eib = eib.where(eib.time.dt.year >= YMIN, drop = True)

In [ ]:
eobs = xr.open_mfdataset(["E-OBS/rr_ens_mean_0.25deg_reg_v31.0e.nc", "E-OBS/fg_ens_mean_0.25deg_reg_v31.0e_remap.nc"]).sel(longitude = slice(-20,40), latitude = slice(32,75), time = slice(str(YMIN), "2024"))

In [ ]:
LON_MIN, LON_MAX, LAT_MIN, LAT_MAX = eobs.longitude.min(), eobs.longitude.max(), eobs.latitude.min(), eobs.latitude.max()
BOX = Polygon((
    (LON_MIN, LAT_MIN),
    (LON_MIN, LAT_MAX),
    (LON_MAX, LAT_MAX),
    (LON_MAX, LAT_MIN)
))
## plot
ax = plt.axes(projection = ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels = True)
ax.plot(*BOX.exterior.xy, color = "red", label = "BIWE")

In [ ]:
# Flag points within the zone
print(len(eib.record))
is_zone = lambda lon, lat : BOX.contains(Point(lon, lat))
eib["in_zone"] = xr.apply_ufunc(np.vectorize(is_zone), eib.lon, eib.lat)
# Filter extended ibtracs
eib = eib.where((eib.time.dt.hour == 0) & # E-OBS is daily
                eib.in_zone.max("dataset"), 
                drop = True) 
print(len(eib.record))

In [ ]:
# Select cyclone times from EOBS field
T = eib.time
eobs_sample = eobs.sel(time = T, method = "nearest") 

In [ ]:
# Merge the track and sst data
M = xr.merge([eib, eobs_sample.drop_vars("time")],)

In [ ]:
M.rr.isel(record = slice(300, 340)).plot(col = "record", col_wrap = 8, cmap = "YlGnBu")

In [ ]:
M.fg.isel(record = slice(300, 340)).plot(col = "record", col_wrap = 8, cmap = "Reds")

In [ ]:
# Save
M.to_netcdf("extended-ibtracs/EU_tracks_with_E-OBS.nc")